# Solve EEG forward problem

In this example, we go through the process of solving EEG forward problem for the model generated in [this example](1_model_creation.ipynb). The result of this process is a `EEGForwardSolution` which can be used for further computations.

## Model loading

First we load the previously generated `FEModel`.

In [1]:
from pathlib import Path
from shamo import FEModel

model_name = "model"
model_path = Path(("./derivatives/1_model_creation/{name}"
                   "/{name}.json").format(name=model_name))
model = FEModel.load(model_path)

Now the model is loaded and we can use it to compute a leadfield matrix.

## Problem definition

In order to generate an `EEGForwardSolution`, we must define an `EEGForwardProblem`.

In [2]:
from shamo import EEGForwardProblem

problem = EEGForwardProblem()

We can now specify the settings of the problem. As we are trying to solve the EEG forward problem, we must set the electrical conductivity of all the tissues present in the model. Two methods can be used:
- `set_electrical_conductivity()` to set σ tissue by tissue.
- `set_electrical_conductivities()` to set multiple σ at once.

> **Note:**  
> Here we set σ [S/m] to some arbitrary values but you should always make sure to use the proper values for the tissues you define.

In [3]:
problem.set_electrical_conductivity("a", 1.0)
sigmas = {"b": 0.5, "c": 0.25}
problem.set_electrical_conductivities(sigmas, {"b": "b_anisotropy"})

{'regions_of_interest': [],
 'markers': [],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': 'b_anisotropy'},
  'c': {'value': 0.25, 'anisotropy': ''}},
 'reference': ''}

Now that the tissues are all setup, we can focus on the sensors. The problem requires a reference which is the ground sensor. We add it by its name in the model.

In [4]:
problem.set_reference("A")

{'regions_of_interest': [],
 'markers': [],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': 'b_anisotropy'},
  'c': {'value': 0.25, 'anisotropy': ''}},
 'reference': 'A'}

We can also list the names of the sensors which should not be considered in the computation. Those are called "markers". We can add them using both:
- `add_marker()` to add only one sensor to the list.
- `add_markers()` to add several sensors at once.

For the sake of this example, we will add two markers.

In [5]:
problem.add_markers(["C", "G"])

{'regions_of_interest': [],
 'markers': ['C', 'G'],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': 'b_anisotropy'},
  'c': {'value': 0.25, 'anisotropy': ''}},
 'reference': 'A'}

Finaly, we must list the tissues for which we want to compute the leadfield matrix. Once again, it is possible to add them either one by one or by batchs using the following methods:
- `add_region_of_interest()`
- `add_regions_of_interest()`

In [6]:
problem.add_regions_of_interest(["b", "c"])

{'regions_of_interest': ['b', 'c'],
 'markers': ['C', 'G'],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': 'b_anisotropy'},
  'c': {'value': 0.25, 'anisotropy': ''}},
 'reference': 'A'}

## Problem resolution

Now that the problem is completely defined, we just have to solve it.

In [7]:
solution = problem.solve("solution", 
                          "./derivatives/2_eeg_forward", model)

## Evaluation

The `EEGForwardSolution` contains a leadfield matrix $L$ that can be used to compute the electrical potentials recorded by the sensors resulting from a source array. This is achieved through the equation:

$$
L \cdot \vec{s} = \vec{r}
$$

To compute a recodings vector $\vec{r}$ using an `EEGForwardSolution`, simply use one of the following methods:
- `evaluate()`
- `evaluate_for_elements()`
- `evaluate_for_sources()`

The simplest is `evaluate_for_sources()` as it only requires you to define an array of `EEGSource` objects. This is what we will use.

> **Note:**  
> In this example, we only define one source but you can use as many as you want.

In [8]:
from shamo import EEGSource

sources = [EEGSource((6, 6, 6), (1, 0, 0), 1e-6)]
solution.evaluate_for_sources(sources)

{'B': 0.00315052392578125,
 'D': 0.00506706787109375,
 'E': -0.0062497861328125,
 'F': -0.00755338818359375,
 'H': -0.009730138671875}